Notebook to perform tests of runtime type checking in MicroPython.


In [13]:
# %pip install micropython-magic nbformat ipykernel mpy-cross

%load_ext micropython_magic

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.6/899.6 KB 1.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
The micropython_magic extension is already loaded. To reload it, use:
  %reload_ext micropython_magic


# Deploy to board

In [11]:
!mpflash list

5l
Updating board info ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00
                        Connected boards                        
┏━━━━━━━┳━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━┓
┃Ser.   ┃Fam.┃Board                        ┃Version        ┃Bld┃
┡━━━━━━━╇━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━┩
│ttyACM1│upy │RPI_PICO                     │v1.24.0-preview│364│
│       │    │Raspberry Pi Pico with RP2040│               │   │
└───────┴────┴─────────────────────────────┴───────────────┴───┘


In [14]:

%run ./deploy.ipynb

Install __future__
Installing __future__ (latest) from https://micropython.org/pi/v2 to /lib
Installing: /lib/__future__.mpy
Done
/mnt/d/mypython/!-stubtestprojects/rt_typing/mp
/mnt/d/mypython/!-stubtestprojects/rt_typing
ls :/
           0 lib/
ls :/lib
         202 __future__.mpy
          70 abc.mpy
           0 collections/
         394 typing.mpy
          66 typing_extensions.mpy
ls :/lib/collections
          82 abc.mpy
True


In [15]:
%mpy import sys;print(f"Testing on {sys.implementation.name} {sys.platform}")

['Testing on micropython rp2']

# Type annotation tests

In [5]:
# %%micropython --reset
import __future__
import typing
import typing_extensions
import abc

True


## module: typing

### typing.protocol

In [6]:
# %%micropython --reset

from typing import Protocol


class Adder(Protocol):
    def add(self, x, y): ...


class IntAdder:
    def add(self, x, y):
        return x + y


class FloatAdder:
    def add(self, x, y):
        return x + y


def add(adder: Adder) -> None:
    print(adder.add(2, 3))


add(IntAdder())
add(FloatAdder())

True
5
5


In [7]:
# %%micropython

from typing import List, Protocol

l: List[int] = [1, 2, 3]


class Speak(Protocol):
    def speak(self): ...


class Parrot:
    def speak(self) -> None:
        print("Polly wants a cracker")


def say_something(speaker: Speak) -> None:
    speaker.speak()


polly = Parrot()

print(say_something(polly))

Polly wants a cracker
None


### typing.NewType

In [11]:
# %%micropython

from typing import NewType

UserId = NewType("UserId", int)
some_id = UserId(524313)

print(some_id)

assert isinstance(some_id, int)

# CPython
# MicroPython typing.py
# ----------------------
# 524313

# Micropython - modtyping.c
# -------------------------
# <any_call>
# [33m[1mWARNING [0m | [33m[1mTraceback (most recent call last):[0m
# [33m[1mWARNING [0m | [33m[1m  File "<stdin>", line 10, in <module>[0m
# [31m[1mERROR   [0m | [31m[1mAssertionError:
# [0m

524313


### typing.Any

In [13]:
# %%micropython

from typing import Any

a: Any = None
a = []  # OK
a = 2  # OK

s: str = ""
s = a  # OK


def foo(item: Any) -> int:
    # Passes type checking; 'item' could be any type,
    # and that type might have a 'bar' method
    item.bar()
    return 42


def hash_b(item: Any) -> int:
    try:
        # Passes type checking
        item.magic()
        return foo(item)
    except AttributeError:
        # just ignore any error for this test
        pass
    return 21
    ...


# Passes type checking, since Any is compatible with all types
print(hash_b(42))
print(hash_b("foo"))

21
21


### typing.AnyString

In [14]:
# %%micropython

from typing import AnyStr


def concat(a: AnyStr, b: AnyStr) -> AnyStr:
    return a + b


concat("foo", "bar")  # OK, output has type 'str'
concat(b"foo", b"bar")  # OK, output has type 'bytes'
try:
    concat("foo", b"bar")  # Error, cannot mix str and bytes
except TypeError as e:
    print("OK, expected:", e)

OK, expected: unsupported types for __add__: 'str', 'bytes'


### typing.LiteralString

In [15]:
# %%micropython

from typing import LiteralString


def run_query(sql: LiteralString) -> None: ...


def caller(arbitrary_string: str, literal_string: LiteralString) -> None:
    run_query("SELECT * FROM students")  # OK
    run_query(literal_string)  # OK
    run_query("SELECT * FROM " + literal_string)  # OK
    run_query(arbitrary_string)  # type checker error
    run_query(f"SELECT * FROM students WHERE name = {arbitrary_string}")  # type checker error

    assert isinstance(literal_string, str), "literal_string should be a string"
    assert isinstance(arbitrary_string, str), "arbitrary_string should be a string"


some_str = "a" * 1000
literal_str = "drop * from tables"

caller(some_str, literal_str)

### typing.reveal_type

In [17]:
# %%micropython

from typing import Self, reveal_type


class Foo:
    def return_self(self) -> Self:
        ...
        return self


class SubclassOfFoo(Foo):
    pass


# reveal type does not work in micropython

print(reveal_type(Foo().return_self()))  # Revealed type is "Foo"
print(reveal_type(SubclassOfFoo().return_self()))  # Revealed type is "SubclassOfFoo"

# Python
# ----------------------
# %% micropython

from typing import Self, reveal_type


class Foo:
    def return_self(self) -> Self:
        ...
        return self


class SubclassOfFoo(Foo):
    pass


# reveal type does not work in micropython

print(reveal_type(Foo().return_self()))  # Revealed type is "Foo"
print(reveal_type(SubclassOfFoo().return_self()))  # Revealed type is "SubclassOfFoo"

# MicroPython typing.py
# ----------------------
# Foo
# <Foo object at 20007850>
# SubclassOfFoo
# <SubclassOfFoo object at 20007860>

# MicroPython typing.py
# ----------------------
# Foo
# <Foo object at 20007850>
# SubclassOfFoo
# <SubclassOfFoo object at 20007860>

# Micropython - modtyping.c
# -------------------------
# <any_call>
# <any_call>
# <any_call>
# <any_call>

Foo
<Foo object at 2000ab90>
SubclassOfFoo
<SubclassOfFoo object at 2000aba0>
Foo
<Foo object at 2000b080>
SubclassOfFoo
<SubclassOfFoo object at 2000b090>


### typing.@no_type_check

In [18]:
# %%micropython

from typing import no_type_check


@no_type_check
def foo(x: int) -> str:
    return x


print(foo("42"))

# MicroPython typing.py
# ----------------------
# 42
# MicroPython typing.py
# ----------------------
# 42

# Micropython - modtyping.c
# -------------------------
# <any_call>

42


In [ ]:
# %%micropython

from typing import no_type_check


@no_type_check
def foo(x: int) -> str:
    return x


print(foo("42"))
assert foo(42) == 42

# CPython
# MicroPython typing.py
# ----------------------
# 42

# Micropython - modtyping.c
# ----------------------
# <any_call>
# WARNING  | Traceback (most recent call last):
# WARNING  |   File "<stdin>", line 12, in <module>
# ERROR    | AssertionError:

### typing.@overload

In [19]:
# %%micropython

from typing import overload


@overload
def bar(x: int) -> str: ...


@overload
def bar(x: str) -> int: ...


def bar(x):
    return x


print(bar(42))

42


### typing.NewType

In [20]:
# %%micropython

from typing import NewType

UserId = NewType("UserId", int)
some_id = UserId(524313)

print(some_id)

assert isinstance(some_id, int)


# MicroPython typing.py
# ----------------------
# 524313


# MicroPython typing.py
# ----------------------
# 524313

# Micropython - modtyping.c
# -------------------------
# <any_call>
# WARNING |mTraceback (most recent call last):
# WARNING |m  File "<stdin>", line 10, in <module>
# ERROR   |mAssertionError:
#

524313


### typing.TypeDict -- TODO

In [19]:
# %%micropython

"""
Tests for basic usage of TypedDict.
"""

from typing import TypeVar, TypedDict


class Movie(TypedDict):
    name: str
    year: int


movie: Movie = {"name": "Blade Runner", "year": 1982}


def record_movie(movie: Movie) -> None:
    ...


record_movie({"name": "Blade Runner", "year": 1982})


movie["director"] = "Ridley Scott"  # E: invalid key 'director'
movie["year"] = "1982"  # E: invalid value type ("int" expected)

# The code below should be rejected, since 'title' is not a valid key,
# and the 'name' key is missing:
movie2: Movie = {"title": "Blade Runner", "year": 1982}  # E

m = Movie(name='Blade Runner', year=1982)


# > TypedDict type objects cannot be used in isinstance() tests such as
# > isinstance(d, Movie).
try: 
    if isinstance(movie, Movie):  # E
        pass
except TypeError as e:
    print("Handled according to spec")

# TypedDict should not be allowed as a bound for a TypeVar.
T = TypeVar("T", bound=TypedDict) # E

Handled according to spec


### typing.TypedDict - Required/NotRequired - FAIL

In [27]:
# %%micropython
# https://github.com/python/typing/blob/main/conformance/tests/typeddicts_required.py
"""
Tests the Required and NotRequired special forms.
"""

# Specification: https://typing.readthedocs.io/en/latest/spec/typeddict.html#required-and-notrequired

from typing import Annotated, NotRequired, Required

from collections.abc import ABC as TypedDict


# Required and NotRequired are valid only within a TypedDict.
class NotTypedDict:
    x: Required[int]  # E: Required not allowed in this context

    def __init__(self, x: int) -> None:
        self.x = x


def func1(
    x: NotRequired[int],  # E: NotRequired not allowed in this context
) -> None:
    pass


class TD1(TypedDict, total=False):
    a: int


class TD2(TD1, total=True):
    b: int


class TD3(TypedDict):
    a: NotRequired[int]
    b: Required[int]


# class TD4(TypedDict, total=False):
#     a: int
#     b: Required[int]


# class TD5(TypedDict, total=True):
#     a: NotRequired[int]
#     b: int


# td3: TD3 = {"b": 0}
# td4: TD4 = {"b": 0}
# td5: TD5 = {"b": 0}

# # These are all equivalent types, so they should be
# # bidirectionally type compatible.
# td3 = td4
# td3 = td5
# td4 = td3
# td4 = td5
# td5 = td3
# td5 = td4


# class TD6(TypedDict):
#     a: Required[Required[int]]  # E: Nesting not allowed
#     b: Required[NotRequired[int]]  # E: Nesting not allowed


# class TD7(TypedDict):
#     # > Required[] and NotRequired[] can be used with Annotated[], in any nesting order.
#     x: Annotated[Required[int], ""]
#     y: Required[Annotated[int, ""]]
#     z: Annotated[Required[Annotated[int, ""]], ""]


# RecursiveMovie = TypedDict(
#     "RecursiveMovie", {"title": Required[str], "predecessor": NotRequired["RecursiveMovie"]}
# )

# movie: RecursiveMovie = {"title": "Beethoven 3", "predecessor": {"title": "Beethoven 2"}}


WARNING  | Traceback (most recent call last):
WARNING  |   File "<stdin>", line 28, in <module>
ERROR    | TypeError: function doesn't take keyword arguments



MCUException: TypeError: function doesn't take keyword arguments


### typing.NamedTuple -- TODO

### typing.TypeVar

In [17]:
# CPython
from typing import List, TypeVar

T = TypeVar("T")


def first(container: List[T]) -> T:
    return container[0]


list_one: List[str] = ["a", "b", "c"]
print(first(list_one))

list_two: List[int] = [1, 2, 3]
print(first(list_two))

a
1


In [16]:
# %%micropython
from typing import List, TypeVar

T = TypeVar("T")


def first(container: List[T]) -> T:
    return container[0]


list_one: List[str] = ["a", "b", "c"]
print(first(list_one))

list_two: List[int] = [1, 2, 3]
print(first(list_two))

a
1


### typing.ParamSpec - TODO:

### typing.Generator

In [21]:
# %%micropython

from typing import Generator


def echo_round() -> Generator[int, float, str]:
    sent = yield 0
    while sent >= 0:
        sent = yield round(sent)
    return "Done"


e = echo_round()
print(next(e))

0


### typing.LiteralString

In [22]:
# %%micropython

from typing import LiteralString


def run_query(sql: LiteralString) -> None: ...


def caller(arbitrary_string: str, literal_string: LiteralString) -> None:
    run_query("SELECT * FROM students")  # OK
    run_query(literal_string)  # OK
    run_query("SELECT * FROM " + literal_string)  # OK
    run_query(arbitrary_string)  # type checker error
    run_query(f"SELECT * FROM students WHERE name = {arbitrary_string}")  # type checker error


some_str = "a" * 1000
literal_str = "drop * from tables"

caller(some_str, literal_str)

### typing.NoReturn

In [23]:
# %%micropython

from typing import NoReturn


def stop() -> NoReturn:
    raise RuntimeError("no way")

In [ ]:
# %%micropython

from typing import NoReturn


def hard_stop() -> NoReturn:
    raise RuntimeError("stop execution here")


print("hello")
hard_stop()

# this line will not be executed - but this is not shown a jupyter notebook
print("world")

### typing.Final

In [24]:
# %%micropython

from typing import Final

CONST: Final = 42


print(CONST)

42


### typing.final

In [25]:
# %%micropython

from typing import final


class Base:
    @final
    def done(self) -> None: ...
class Sub(Base):
    def done(self) -> None:  # Error reported by type checker
        ...


@final
class Leaf: ...


class Other(Leaf):  # Error reported by type checker
    ...


other = Other()

### typing.TypedDict - TODO

### typing.NamedTuple  - TODO

## typing.IO - TODO 
- ABCs for working with IO - Need tests

In [ ]:
# %%micropython

# ABCs for working with IO
# Generic type IO[AnyStr] and its subclasses TextIO(IO[str]) and BinaryIO(IO[bytes]) represent the types of I/O streams such as returned by open().

from typing import IO
from typing import TextIO
from typing import BinaryIO

print("TODO: Add some tests")

TODO: Add some tests


## Module typing_extensions

This is relevant to MicroPython as it is based on 3.4 / 3.5 syntax.

Enable use of new type system features on older Python versions. For example, typing.TypeGuard is new in Python 3.10, but typing_extensions allows users on previous Python versions to use it too.


see: https://typing-extensions.readthedocs.io/

### typing_extensions.TypeVarTuple

In [26]:
# %%micropython

# In older versions of Python, TypeVarTuple and Unpack
# are located in the `typing_extensions` backports package.
from typing_extensions import TypeVarTuple, Unpack

Ts = TypeVarTuple("Ts")
tup: tuple[Unpack[Ts]]  # Semantically equivalent, and backwards-compatible

### typing_extensions.TypeVar

In [27]:
# %%micropython

from typing_extensions import TypeVar, reveal_type

Self = TypeVar("Self", bound="Foo")


class Foo:
    def return_self(self: Self) -> Self:
        ...
        return self


foo = Foo()
reveal_type(foo.return_self())  # Revealed type is "Foo"

Foo


### typing_extensions.Generator

In [ ]:
# %%micropython

from typing_extensions import Generator


def echo_round() -> Generator[int, float, str]:
    sent = yield 0
    while sent >= 0:
        sent = yield round(sent)
    return "Done"


e = echo_round()

output = next(e)
print(output)
assert output == 0

0


### typing_extensions.Self

In [28]:
# %%micropython
from typing_extensions import Self, reveal_type


class Foo:
    def return_self(self) -> Self:
        ...
        return self


class SubclassOfFoo(Foo):
    pass


print(reveal_type(Foo().return_self()))  # Revealed type is "Foo"
print(reveal_type(SubclassOfFoo().return_self()))  # Revealed type is "SubclassOfFoo"

# Same issues as with : from typing import reveal_type
# impact: Low 

Foo
<Foo object at 2000e950>
SubclassOfFoo
<SubclassOfFoo object at 2000e960>


## module: `__future__`

### __future__.annotations

In [29]:
# %%micropython

from __future__ import annotations

from typing import cast
from typing_extensions import reveal_type

x = 1
reveal_type(x)
y = cast(str, x)
reveal_type(y)

try:
    y.upper()
except AttributeError as e:
    print("OK, Expected error:", e)

int
int
OK, Expected error: 'int' object has no attribute 'upper'


In [30]:
# %%micropython

from __future__ import annotations

from typing import cast
from typing_extensions import reveal_type

x = 1
reveal_type(x)
y = cast(str, x)
reveal_type(y)
try:
    y.upper()
except AttributeError as e:
    # https://docs.python.org/3/library/typing.html#typing.cast
    print(f"OK - Intentional no runtime check: {e}")

int
int
OK - Intentional no runtime check: 'int' object has no attribute 'upper'


In [31]:
# %%micropython

from __future__ import annotations

from typing import cast
from typing_extensions import reveal_type

x = 1
reveal_type(x)
y = cast(str, x)
reveal_type(y)
try:
    y.upper()
except AttributeError as e:
    # https://docs.python.org/3/library/typing.html#typing.cast
    print(f"OK - Intentional no runtime check: {e}")

int
int
OK - Intentional no runtime check: 'int' object has no attribute 'upper'


## module: abc


In [32]:
# %%micropython

from abc import get_cache_token, update_abstractmethods, ABC, abstractmethod


class C(ABC):
    @abstractmethod
    def my_abstract_method(self, arg1): ...
    @classmethod
    @abstractmethod
    def my_abstract_classmethod(cls, arg2): ...
    @staticmethod
    @abstractmethod
    def my_abstract_staticmethod(arg3): ...

    @property
    @abstractmethod
    def my_abstract_property(self): ...
    @my_abstract_property.setter
    @abstractmethod
    def my_abstract_property(self, val): ...

    @abstractmethod
    def _get_x(self): ...
    @abstractmethod
    def _set_x(self, val): ...

    x = property(_get_x, _set_x)


token = get_cache_token()

cls = update_abstractmethods(C)

### abc.abstractmethod

In [ ]:
# %%micropython --reset
from typing import TYPE_CHECKING


from abc import ABC, abstractmethod
from math import pi


class Shape(ABC):
    @abstractmethod
    def get_area(self) -> float:
        pass

    @abstractmethod
    def get_perimeter(self) -> float:
        pass


class Circle(Shape):
    def __init__(self, radius) -> None:
        self.radius = radius

    def get_area(self) -> float:
        return pi * self.radius**2

    def get_perimeter(self) -> float:
        return 2 * pi * self.radius


class Square(Shape):
    def __init__(self, side) -> None:
        self.side = side

    def get_area(self) -> float:
        return self.side**2

    def get_perimeter(self) -> float:
        return 4 * self.side


c1 = Circle(5)
s1 = Square(5)

for shape in [c1, s1]:
    a = shape.get_area()
    p = shape.get_perimeter()

    print(a, p)
    print(f"{type(a)=}")
    print(f"{type(p)=}")

    assert isinstance(a, (float, int)), "Area should be a float"
    assert isinstance(p, (float, int)), "Perimeter should be a float"

# CPython
# ----------------------
# 78.53981633974483 31.41592653589793
# type(a)=<class 'float'>
# type(p)=<class 'float'>
# 25 20
# type(a)=<class 'int'>
# type(p)=<class 'int'>

# MicroPython typing.py
# ----------------------
# 78.53982 31.41593
# type(a)=<class 'float'>
# type(p)=<class 'float'>
# 25 20
# type(a)=<class 'int'>
# type(p)=<class 'int'>

# Micropython - modtyping.c
# ----------------------
# <any_call> <any_call>
# type(a)=<class 'any_call'>
# type(p)=<class 'any_call'>
# WARNING |Traceback (most recent call last):
# WARNING |  File "<stdin>", line 52, in <module>
# ERROR   |AssertionError: Area should be a float
#

True
78.53982 31.41593
type(a)=<class 'float'>
type(p)=<class 'float'>
25 20
type(a)=<class 'int'>
type(p)=<class 'int'>


In [33]:
# %%micropython

from abc import ABC, abstractmethod
from math import pi


class Shape(ABC):
    @abstractmethod
    def get_area(self) -> float:
        pass

    @abstractmethod
    def get_perimeter(self) -> float:
        pass


class Circle(Shape):
    def __init__(self, radius) -> None:
        self.radius = radius

    def get_area(self) -> float:
        return pi * self.radius**2

    def get_perimeter(self) -> float:
        return 2 * pi * self.radius


class Square(Shape):
    def __init__(self, side) -> None:
        self.side = side

    def get_area(self) -> float:
        return self.side**2

    def get_perimeter(self) -> float:
        return 4 * self.side

## collections.abc


### collections.abc.Mapping

In [34]:
# %%micropython
# https://docs.python.org/3/library/typing.html#generics

from collections.abc import Mapping, Sequence


class Employee: ...


# Sequence[Employee] indicates that all elements in the sequence
# must be instances of "Employee".
# Mapping[str, str] indicates that all keys and all values in the mapping
# must be strings.
def notify_by_email(employees: Sequence[Employee], overrides: Mapping[str, str]) -> None: ...

In [ ]:
# %%micropython


from collections.abc import Mapping

# Type checker will infer that all elements in ``x`` are meant to be ints
x: list[int] = []

# Type checker error: ``list`` only accepts a single type argument:
y: list[int, str] = [1, "foo"]

# Type checker will infer that all keys in ``z`` are meant to be strings,
# and that all values in ``z`` are meant to be either strings or ints
z: Mapping[str, str | int] = {}

print(x, y, z)

[] [1, 'foo'] {}


### collections.abc.Sequence

In [ ]:
# %%micropython


from collections.abc import Mapping

# Type checker will infer that all elements in ``x`` are meant to be ints
x: list[int] = []

# Type checker error: ``list`` only accepts a single type argument:
y: list[int, str] = [1, "foo"] 

# Type checker will infer that all keys in ``z`` are meant to be strings,
# and that all values in ``z`` are meant to be either strings or ints
z: Mapping[str, str | int] = {}

In [ ]:
# %%micropython

from collections.abc import Mapping, Sequence


class Employee: ...


# Sequence[Employee] indicates that all elements in the sequence
# must be instances of "Employee".
# Mapping[str, str] indicates that all keys and all values in the mapping
# must be strings.
def notify_by_email(employees: Sequence[Employee], overrides: Mapping[str, str]) -> None: ...

### collections.abc.Callable

In [35]:
# %%micropython

# ParamSpec, 3.11 notation
# https://docs.python.org/3/library/typing.html#typing.ParamSpec

from collections.abc import Callable
from typing import TypeVar, ParamSpec

T = TypeVar("T")
P = ParamSpec("P")


def add_logging(f: Callable[P, T]) -> Callable[P, T]:
    """A type-safe decorator to add logging to a function."""

    def inner(*args: P.args, **kwargs: P.kwargs) -> T:
        print(f"{f.__name__} was called")
        return f(*args, **kwargs)

    return inner


@add_logging
def add_two(x: float, y: float) -> float:
    """Add two numbers together."""
    return x + y


x = add_two(1, 2)
print(x)
assert x == 3, "add_two(1, 2) == 3"

add_two was called
3


In [ ]:
# %%micropython

# ParamSpec, 3.11 notation
# https://docs.python.org/3/library/typing.html#typing.ParamSpec

from collections.abc import Callable
from typing import TypeVar, ParamSpec

T = TypeVar("T")
P = ParamSpec("P")


def add_logging(f: Callable[P, T]) -> Callable[P, T]:
    """A type-safe decorator to add logging to a function."""

    def inner(*args: P.args, **kwargs: P.kwargs) -> T:
        print(f"{f.__name__} was called")
        return f(*args, **kwargs)

    return inner


@add_logging
def add_two(x: float, y: float) -> float:
    """Add two numbers together."""
    return x + y


x = add_two(1, 2)
print(x)

WARNING  | Traceback (most recent call last):
WARNING  |   File "<stdin>", line 6, in <module>
ERROR    | ImportError: no module named 'collections.abc'



MCUException: ImportError: no module named 'collections.abc'


### collections.abc.Awaitable

In [62]:
# %%micropython
from collections.abc import Callable, Awaitable


def feeder(get_next_item: Callable[[], str]) -> None: ...  # Body


def async_query(
    on_success: Callable[[int], None], on_error: Callable[[int, Exception], None]
) -> None: ...  # Body


async def on_update(value: str) -> None: ...  # Body


callback: Callable[[str], Awaitable[None]] = on_update

# ...


def concat(x: str, y: str) -> str:
    return x + y


x: Callable[..., str]
x = str  # OK
x = concat  # Also OK



In [ ]:
# %%micropython
from collections.abc import Callable, Awaitable

import asyncio


async def blink(led, period_ms):
    while True:
        await asyncio.sleep_ms(5)
        print(f"fake {led} ON")
        await asyncio.sleep_ms(period_ms)
        print(f"fake {led} OFF")


async def work(todo: List[Callable[[], Awaitable[None]]], timeout_ms: int) -> None:
    for task in todo:
        asyncio.create_task(task())
    await asyncio.sleep_ms(timeout_ms)


async def main():
    await work(
        [
            lambda: blink("LED1", 70),
            lambda: blink("led2", 20),
        ],
        90,
    )


try:
    asyncio.run(main())
finally:
    asyncio.new_event_loop()  # Clear retained stat

fake LED1 ON
fake led2 ON
fake led2 OFF
fake led2 ON
fake led2 OFF
fake led2 ON
fake LED1 OFF
fake led2 OFF
fake LED1 ON
fake led2 ON


### collections.abc.Iterable

In [ ]:

# ####

from collections.abc import Iterable
from typing import Protocol


class Combiner(Protocol):
    def __call__(self, *vals: bytes, maxlen: int | None = None) -> list[bytes]: ...


def batch_proc(data: Iterable[bytes], cb_results: Combiner) -> bytes:
    for item in data:
        ...


def good_cb(*vals: bytes, maxlen: int | None = None) -> list[bytes]: ...


batch_proc([], good_cb)  # OK

## asyncio

In [ ]:
# %%micropython
from collections.abc import Callable, Awaitable

import asyncio


async def blink(led, period_ms):
    while True:
        await asyncio.sleep_ms(5)
        print(f"fake {led} ON")
        await asyncio.sleep_ms(period_ms)
        print(f"fake {led} OFF")


async def work(todo: List[Callable[[], Awaitable[None]]], timeout_ms: int) -> None:
    for task in todo:
        asyncio.create_task(task())
    await asyncio.sleep_ms(timeout_ms)


async def main():
    await work(
        [
            lambda: blink("LED1", 70),
            lambda: blink("led2", 20),
        ],
        90,
    )


try:
    asyncio.run(main())
finally:
    asyncio.new_event_loop()  # Clear retained stat

fake LED1 ON
fake led2 ON
fake led2 OFF
fake led2 ON
fake led2 OFF
fake led2 ON
fake LED1 OFF
fake led2 OFF
fake LED1 ON
fake led2 ON


# Unsupported constructs

### typing.get_args - Not Implemented

In [47]:
# %%micropython
from typing import get_args

# partial implementation of get_args
x = get_args(int)
assert x == (), f"expected () but got {x}"
# assert get_args(Dict[int, str]) == (int, str)
# assert get_args(Union[int, str]) == (int, str)

### typing.get_origin - Not Implemented

In [ ]:
# %%micropython

from typing import Dict,get_origin


# https://docs.python.org/3/library/typing.html#typing.get_origin

assert get_origin(str) is None, "str"

# Partial implementation of get_origin

assert get_origin(Dict[str, int]) is dict, "origin Dict cannot be detected"
# assert get_origin(Union[int, str]) is Union, "Union"
# P = ParamSpec("P")
# assert get_origin(P.args) is P, "ParamSpec args"
# assert get_origin(P.kwargs) is P, "ParamSpec kwargs"

WARNING  | Traceback (most recent call last):
WARNING  |   File "<stdin>", line 12, in <module>
ERROR    | AssertionError: origin Dict cannot be detected



MCUException: AssertionError: origin Dict cannot be detected


### typing.@runtime_checkable - Not Implemented

In [48]:
# %%micropython

from typing import runtime_checkable, Protocol


@runtime_checkable
class Closable(Protocol):
    def close(self): ...


try:
    assert isinstance(open("lib/typing.mpy"), Closable)
except TypeError as e:
    print(f"@runtime_checkable, not supported: {e}")

@runtime_checkable, not supported: issubclass() arg 2 must be a class or a tuple of classes


### abc.ABCMeta - `metaclass` Not Implemented

In [49]:
# %%micropython

from abc import ABCMeta


class MyABC(metaclass=ABCMeta):
    pass

WARNING  | Traceback (most recent call last):
WARNING  |   File "<stdin>", line 6, in <module>
ERROR    | TypeError: function doesn't take keyword arguments



MCUException: TypeError: function doesn't take keyword arguments


## python 3.12 syntax - not supported


### python_3.12 type

In [51]:
# %%micropython

# 3.12 type parameter syntax
# https://docs.python.org/3/reference/simple_stmts.html#the-type-statement


print("OK, 3.12 syntax not supported")
type Point = tuple[float, float]


WARNING  | Traceback (most recent call last):
  File "<stdin>", line 8
ERROR    | SyntaxError: invalid syntax



MCUException: SyntaxError: invalid syntax


### python_3.12 type parameter sytax

In [52]:
# %%micropython

# 3.12 type parameter syntax not supported

from collections.abc import Sequence

def first[T](l: Sequence[T]) -> T:  # Function is generic over the TypeVar "T"
    return l[0]

# from collections.abc import Sequence
# from typing import TypeVar

# U = TypeVar('U')                  # Declare type variable "U"

# def second(l: Sequence[U]) -> U:  # Function is generic over the TypeVar "U"

print("OK, 3.12 syntax not supported")

WARNING  | Traceback (most recent call last):
  File "<stdin>", line 7
ERROR    | SyntaxError: invalid syntax



MCUException: SyntaxError: invalid syntax


In [53]:
print("Completed successfully")

Completed successfully
